In [5]:
from io import BytesIO
from ladybug.epw import EPW
from ladybug.location import Location
from ladybug.sunpath import Sunpath
from ladybug_comfort.collection.utci import UTCI
from zipfile import ZipFile
import geopandas as gpd
import ladybug_pandas as lbp
import numpy as np
import pandas as pd
import re
import requests

In [30]:
# https://climate.onebuilding.org/WMO_Region_4_North_and_Central_America/default.html
df = gpd.read_file(
    "Region4_USA_TMYx_EPW_Processing_locations.kml", driver="KML", header=0)
df.drop(df[df["Description"] == ""].index, inplace=True)

data = pd.DataFrame()
data["name"] = df["Name"]
data["latitude"] = df["geometry"].apply(lambda x: x.y)
data["longitude"] = df["geometry"].apply(lambda x: x.x)

df.head()

data["url"] = df["Description"].apply(
    lambda x: re.search("http.*?\.zip", x).group(0))
data.drop(data[data["url"].apply(
    lambda x: "TMYx.zip" not in x)].index, inplace=True)

data.index = data["url"].apply(
    lambda x: re.search(".*\.(.+?)_TMYx", x).group(1))
data.index.name = "wmo"
data.head()

<>:14: SyntaxWarning: invalid escape sequence '\.'
<>:19: SyntaxWarning: invalid escape sequence '\.'
<>:14: SyntaxWarning: invalid escape sequence '\.'
<>:19: SyntaxWarning: invalid escape sequence '\.'
/tmp/ipykernel_469892/2746786175.py:14: SyntaxWarning: invalid escape sequence '\.'
  lambda x: re.search("http.*?\.zip", x).group(0))
/tmp/ipykernel_469892/2746786175.py:19: SyntaxWarning: invalid escape sequence '\.'
  lambda x: re.search(".*\.(.+?)_TMYx", x).group(1))


,name,latitude,longitude,url
wmo,,,,
690070,Marina.Muni.AP CA USA,36.683,-121.767,https://climate.onebuilding.org/WMO_Region_4_N...
690150,Twentynine.Palms.SELF CA USA,34.300,-116.167,https://climate.onebuilding.org/WMO_Region_4_N...
690170,Indian.Springs.Range.63 NV USA,36.533,-115.567,https://climate.onebuilding.org/WMO_Region_4_N...
690290,Port.Aransas-Mustang.Beach.AP TX USA,27.833,-97.067,https://climate.onebuilding.org/WMO_Region_4_N...
690310,Yuma.Proving.Ground AZ USA,32.833,-114.400,https://climate.onebuilding.org/WMO_Region_4_N...


In [31]:
import time

data["summer"] = np.nan
data["winter"] = np.nan

for i, row in data.iterrows():
    r = requests.get(row["url"])
    zf = ZipFile(BytesIO(r.content))
    namelist = zf.namelist()
    epw_file = [f for f in namelist if "epw" in f][0]
    zf.extract(epw_file, "epws")
    epw_file = row["url"].split("/")[-1].replace(".zip", ".epw")

    epw_data = EPW("epws/" + epw_file)
    utci = UTCI.from_epw(epw_data, include_wind=True, include_sun=True)

    df = pd.Series(utci.is_comfortable.values,
                   index=utci.is_comfortable.datetimes)
    df.drop(df[df.index.hour < 8].index, inplace=True)

    data.loc[i, "summer"] = df["2017-06-21":"2017-09-20"].mean()
    data.loc[i, "winter"] = pd.concat(
        (df["2017-01-01":"2017-03-20"], df["2017-12-21":"2017-12-31"])).mean()

    time.sleep(0.1)

data.drop("url", axis=1, inplace=True)

In [32]:
chicago = data.loc["997338", ["winter", "summer"]]

data["winter_score"] = (data["winter"] / chicago["winter"]).apply(np.log2)
data["summer_score"] = (data["summer"] / chicago["summer"]).apply(np.log2)
data.sort_values(by="winter_score", ascending=False, inplace=True)

In [33]:
print(data.to_string())
data.to_csv("weather.csv")

                                                                                          name  latitude  longitude    summer    winter  winter_score  summer_score
wmo                                                                                                                                                                
722934                                                                Oceanside.Muni.AP CA USA  33.21900 -117.34900  0.456522  0.820139      6.205793     -0.429457
747850                                            McClellanville-Santee.Coastal.Reserve SC USA  33.15300  -79.36400  0.082201  0.786111      6.144658     -2.902911
720743                                                          South.Padre.Island.Heli TX USA  26.07100  -97.15700  0.088315  0.757639      6.091435     -2.799406
994050                                                                  Lake.Worth.Pier FL USA  26.61270  -80.03388  0.075408  0.757639      6.091435     -3.027358
747470          